In [1]:
import sys
sys.path.insert(1, '../../functions/')

from settings import runcry

from crystal_io import read_input
from crystal_io import write_bs
from crystal_io import write_input
from extract_info import band_gap


from extract_info import final_energy

from settings import clean_wf

import re
import numpy as np
from os.path import join
import pandas as pd
import time

In [2]:
#directory where the input is saved
directory = '../data' 

#name of the input (including the extension)
input_name = 'mgo.d12' 

file_path = join(directory,input_name)

#the basis sets are specified as a list. If the input contains more than one elements, then this is a list of lists.
#In this case, the first list contains all the basis sets for the first element and so on.
#All the lists must be of the same lenght.
basis_set = [['Mg_8-511G_harrison_1994','O_8-411_muscat_1999'],['Mg_8-511d1G_valenzano_2006','O_8-411d11G_valenzano_2006']]

#Specify if this is a spin polarised calculation
spin_pol = False

#Run the calculation? (True = run , False = only write the inputs, 'Analyse' = analyse results of a previous calculation)
run = True

#Delete the wave function files after running the calculation
clean = True

In [3]:
#Read the input and store it in its several blocks
geom_block,optgeom_block,bs_block,func_block,scf_block = read_input(file_path)

In [4]:
#This is the function that runs the calculation (move to the following cell to analyse the results)
def bs(directory,input_name,geom_block,func_block,scf_block,basis_set,spin_pol,run,clean):
    files_not_found = []
    energy = []
    for i,basis in enumerate(basis_set):
        input_convergence = join(directory,input_name[:-4]+'_%s.d12'%(i))    
        if run != 'Analyse':
            bs_block = write_bs(basis)              
            write_input(input_convergence,geom_block,bs_block,func_block,scf_block) 
        if run == True:
            time_0 = time.time()
            runcry(input_convergence[:-4])
            time.sleep(3)
            if clean == True:
                clean_wf(directory,input_convergence)  
            E_final = final_energy(input_convergence[:-3]+'out')
            Band_gap = band_gap(input_convergence[:-3]+'out',spin_pol)
            if type(E_final) == float:   
                energy.append([' '.join(basis),float(E_final),float(Band_gap),time.time()-time_0]) 
            else:
                files_not_found.append(input_name[:-4]+'_%s.out'%(i)) 
                #elif type(functional) == list:
                    #files_not_found.append(input_name[:-4]+'_%s_%s.out'%(functional[0]+'_'+functional[1])) 
        if run == 'Analyse':    
            E_final = final_energy(input_convergence[:-3]+'out')
            Band_gap = band_gap(input_convergence[:-3]+'out',spin_pol)
            if type(E_final) == float:   
                energy.append([' '.join(basis),float(E_final),float(Band_gap)]) 
    if run == True:
        energy_arr = np.array(energy)
        energy_df = pd.DataFrame(energy, columns =['Basis set','Energy','Band Gap (eV)','Time'])
        energy_df.set_index('Basis set', inplace=True)
        pd.set_option('display.float_format', '{:.3E}'.format)
    if run == 'Analyse':
        energy_arr = np.array(energy)
        energy_df = pd.DataFrame(energy, columns =['Basis set','Energy','Band Gap (eV)'])
        energy_df.set_index('Basis set', inplace=True)
        pd.set_option('display.float_format', '{:.3E}'.format)
        
        if len(files_not_found) > 0:
            print('The following outputs were not found: '+','.join(files_not_found))
    
    if run != False:
        return energy_df, energy_arr
    else:
        return 'Inputs saved'

### The result is in the cells below

In [5]:
#call the bs function to run the calcaulation/prepare the inputs/analyse the results
#The final energy and band gap is saved as a Pandas dataframe (displayed in the cell below) and as a numpy array.
if run != False:
    energy_df, energy_arr = bs(directory,input_name,geom_block,func_block,scf_block,basis_set,spin_pol,run,clean)
    display(energy_df)
else:
    display(bs(directory,input_name,geom_block,func_block,scf_block,basis_set,spin_pol,run,clean))

,Energy,Band Gap (eV),Time
Basis set,,,
Mg_8-511G_harrison_1994 O_8-411_muscat_1999,-7.495E+03,7.076E+00,1.159E+01
Mg_8-511d1G_valenzano_2006 O_8-411d11G_valenzano_2006,-7.496E+03,7.241E+00,2.822E+01


The input and output files are saved as input_name_n (where input_name is the name of the input specified and n is a number)